# Some potential audiences are:

1. Homeowners who want to increase the sale price of their homes through home improvement projects
2. Advocacy groups who want to promote affordable housing
3. Local elected officials who want to understand how their policy ideas (e.g. zoning changes, permitting) might impact home prices
4. Real estate investors looking for potential "fixer-uppers" or "tear-downs"

# Three things to be sure you establish during this phase are:

1. **Objectives:** what questions are you trying to answer, and for whom?
2. **Project plan:** you may want to establish more formal project management practices, such as daily stand-ups or using a Trello board, to plan the time you have remaining. Regardless, you should determine the division of labor, communication expectations, and timeline.
3. **Success criteria:** what does a successful project look like? How will you know when you have achieved it?

# READ THIS: Import the following data files from https://info.kingcounty.gov/assessor/DataDownload/default.aspx
## Download the files to local repo data directory
> 1) Real Property Sales (.ZIP, csv) <BR>
> 2) Parcel (.ZIP, csv) <BR>
> 3) Residential Building (.ZIP, csv) <BR>
> 4) Unit Breakdown (.ZIP)<BR>


In [1]:
import os
import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from eda.eda import *
from lr_model.build_lr import *

In [2]:
# A function named parse_2019(df) that takes a dataframe as the input.
# It takes in a dataframe, looks for relevant columns, and then keeps the rows that
# are in the year 2019.

def parse_2019(df):
    if 'DocumentDate' in df.columns:
        df = df[pd.to_datetime(df['DocumentDate']).dt.year == 2019 ]
    elif 'ChangeDate' in df.columns:
        df = df[df['ChangeDate'].astype(str).str[:4] == '2019']
    df.reset_index(drop = True)
    return df

In [3]:
## Create a function get_data(create_csv)
## If create_csv = True:
##.   create a combined file rp_cons.csv from other csv files and return a dataframe rp_cons
## If create_csv = False:
##    return a data_frame with all columns from rp_cons.csv

import pandas as pd

def get_data (create_csv):

    if create_csv == False:    
        rp_cons = pd.read_csv("data/rp_cons.csv")   
    return rp_cons
    
    df_rp_sales = get_sale()
    df_parcel = get_parcel()
    df_res_bldg = get_resBldg()
    df_unit_breakdown = get_unit_breakdown()
    
    
    return df_rpsales

### 1) Read EXTR_RPSale.csv 

In [ ]:
# Data File: EXTR_RPSale.csv -------------------------------------------------------------
#Table: EXTR_RPSale 
#Keys: Major, Minor
#Fields: SalePrice, PropertyType, PrincipalUse
def get_sale():
    df_rp_sales = pd.read_csv('/data/EXTR_RPSale.csv', encoding = "ISO-8859-1", low_memory=False)
    print("Before Filer EXTR_RPSale.csv: ", df_rp_sales.shape)

    # Filter the following columns from EXTR_RPsale table
    # Primary key: 'Major', 'Minor' 
    # Select Fields: 'DocumentDate', 'SalePrice', 'PropertyType', 'PrincipalUse', 'PropertyClass
    cols = list(df_rp_sales.columns)
    df_rp_sales = df_rp_sales[cols[1:5] + cols[14:16] + cols[22:23]]
    df_rp_sales = parse_2019(df_rp_sales)
    print("After Filer EXTR_RPSale.csv: ", df_rp_sales.shape)  
    return df_rp_sales

### 2) Read EXTR_Parcel.csv 

In [19]:
#Data File: EXTR_Parcel.csv
#Table: EXTR_Parcel
#Keys: Major, Minor
#Fields: PropType, Area, SubArea,DistrictName, SqFtLot, WaterSystem, SewerSystem, Access, WaterProblems, AirportNoise, TrafficNoise,PowerLines,  LandSlideHazard, SeismicHazard

def get_parcel():
    df_parcel = pd.read_csv('/data/EXTR_Parcel.csv', encoding = "ISO-8859-1", low_memory=False)
    print("Before EXTR_Parcel.csv: ", df_parcel.shape)
    df_parcel.columns
    
    # Filter the following columns from EXTR_Parcel table
    # Primary key: 'Major', 'Minor' 
    # Select Fields: PropType, Area, SubArea, DistrictName, SqFtLot, WaterSystem, SewerSystem, Access, WaterProblems, AirportNoise, TrafficNoise,PowerLines,  LandSlideHazard, SeismicHazard
    cols = list(df_parcel.columns)
    df_parcel = df_parcel[cols[:2] + cols[10:11] + cols[15:16]]  ######## Change this
    df_parcel = parse_2019(df_parcel)
    print("After Filer EXTR_Parcel.csv: ", df_parcel.shape)
    return df_parcel

### 3) Read EXTR_ResBldg.csv 

In [20]:
#Data File: EXTR_ResBldg.csv
#Table: EXTR_ResBldg
#Keys: Major, Minor
#Fields: BldgNbr, NbrLivingUnits, Address, BuildingNumber, Stories, BldgGrade, SqFt1stFloor, SqFtHalfFloor, SqFt2ndFloor, SqFtUpperFloor, SqFtTotLiving, SqFtTotBasement, SqFtFinBasement, SqFtOpenPorch, SqFtEnclosedPorch, SqFtDeck, HeatSystem, HeatSource, Bedrooms, BathHafCouunt, Bath3qtrCount, BathFullCount, FpSingleStory, FpMultiStory, YrBuilt, YrRenovated 
def get_resBldg():
    df_res_bldg = pd.read_csv('/data/EXTR_ResBldg.csv', encoding = "ISO-8859-1", low_memory=False)
    print("Before EXTR_ResBldg.csv: ", df_res_bldg.shape)

    # Filter the following columns from EXTR_Parcel table
    # Primary key: 'Major', 'Minor' 
    # Select Fields: PropType, Area, SubArea, DistrictName, SqFtLot, WaterSystem, SewerSystem, Access, WaterProblems, AirportNoise, TrafficNoise,PowerLines,  LandSlideHazard, SeismicHazard
    cols = list(df_parcel.columns)
    df_res_bldg = df_res_bldg[cols[:2] + cols[10:11] + cols[15:16]]  ######## Change this    
    df_res_bldg = parse_2019(df_res_bldg)
    print("After Filer EXTR_Parcel.csv: ", df_res_bldg.shape)
    return df_res_bldg

### 4) Read EXTR_UnitBreakdown.csv 

In [21]:
#Data File: EXTR_UnitBreakdown.csv
#Table: EXTR_UnitBreakdown
#Keys: Major, Minor
#Fields:  'UnitTypeItemId', 'NbrThisType', 'SqFt','NbrBedrooms', 'NbrBaths'
def get_unitbreakdown():
    df_unit_breakdown = pd.read_csv('/data/EXTR_UnitBreakdown.csv', encoding = "ISO-8859-1", low_memory=False)
    print("EXTR_UnitBreakdown: ", df_unit_breakdown.shape)
    ###todo extract column
    return df_unit_breakdown

### Andrew's scratchwork below:
____

In [ ]:
df_rp_sales = get_sale()

In [9]:
df_rp_sales.columns

NameError: name 'df_rp_sales' is not defined

In [ ]:
df_par.columns

In [ ]:
df_res_bldg.columns

In [ ]:
df_value_history.columns

In [ ]:
df_unit_breakdown.columns

In [10]:
df = df_rp_sales.head()

NameError: name 'df_rp_sales' is not defined

In [11]:
df['DocumentDate'] = pd.to_datetime(df['DocumentDate'])

NameError: name 'df' is not defined

In [12]:
df[df['DocumentDate'].dt.year == 2019 ]

NameError: name 'df' is not defined

In [ ]:
df['DocumentDate'] = pd.to_datetime(df['DocumentDate'])
df[df['DocumentDate'].dt.year == 2019 ]

In [ ]:
df_value_history['ChangeDate'][0][:4]

In [ ]:
df_value_history = df_value_history[df_value_history['ChangeDate'].astype(str).str[:4] == '2019']
df_value_history.reset_index(drop = True)

In [13]:
df = get_sale()

FileNotFoundError: [Errno 2] File b'../data/EXTR_RPSale.csv' does not exist: b'../data/EXTR_RPSale.csv'

In [ ]:
df = df.reset_index(drop = True)

In [ ]:
df.columns

In [14]:
corr = df.corr()
# The mask is not necessary, but corr() has duplicate values on either side of the diagonal
# # half the heatmap is hidden since the top half is redundant
mask = np.triu(np.ones_like(corr, dtype=np.bool))

fig1, ax1 = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, mask=mask, ax=ax1, cmap="viridis");

NameError: name 'df' is not defined

In [ ]:
# # create a smaller df to save space and processing power
fsm_df = df[['DocumentDate', 'SalePrice', 'PropertyType', 'PrincipalUse', 'PropertyClass']].copy()
fsm_df.dropna(inplace=True)
fsm = ols(formula="SalePrice ~ DocumentDate + SalePrice + PropertyType + PrincipalUse + PropertyClass", data=fsm_df)
fsm_results = fsm.fit()

In [ ]:
fsm_results.summary()

In [ ]:
rainbow_statistic, rainbow_p_value = linear_rainbow(fsm_results)
print("Rainbow statistic:", rainbow_statistic)
print("Rainbow p-value:", rainbow_p_value)

In [ ]:
y = fsm_df["SalePrice"]
y_hat = fsm_results.predict()

In [ ]:
fig2, ax2 = plt.subplots()
ax2.set(xlabel="Predicted Life Expectancy",
        ylabel="Residuals (Actual - Predicted Life Expectancy)")
ax2.scatter(x=y_hat, y=y-y_hat, color="blue", alpha=0.2);

In [ ]:
lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(y-y_hat, fsm_df[["Schooling"]])
print("Lagrange Multiplier p-value:", lm_p_value)
print("F-statistic p-value:", f_p_value)

In [ ]:
df_rp_sales = get_sale()

In [ ]:
df_parcel = get_parcel()

In [ ]:
df_res_bldg = get_resBldg()


In [ ]:
df_unit_breakdown = get_unitbreakdown()

In [ ]:
# propertyclass
df_rp_sales.corr()

In [ ]:
df_parcel

In [ ]:
df_res_bldg.corr()

In [ ]:
def maj_min(df, drop=True):
    try:
        df.insert(0, 'Major+Minor', df.Major.astype(str) + df.Minor.astype(str))
        if drop==True:
            df.drop(['Major', 'Minor'], axis=1, inplace=True)
    except:
        print('columns missing')

    return df

In [ ]:
# # create a smaller df to save space and processing power
fsm_df = df[['PropertyClass', 'NbrThisType', 'SqFt', 'NbrBaths', 'SalePrice']].copy()
fsm_df.dropna(inplace=True)
fsm = ols(formula="SalePrice ~ PropertyClass + NbrThisType + SqFt + NbrBaths", data=fsm_df)
fsm_results = fsm.fit()

In [ ]:
fsm_results.summary()

In [ ]:
rainbow_statistic, rainbow_p_value = linear_rainbow(fsm_results)
print("Rainbow statistic:", rainbow_statistic)
print("Rainbow p-value:", rainbow_p_value)

In [ ]:
y = fsm_df["SalePrice"]
y_hat = fsm_results.predict()

In [ ]:
fig2, ax2 = plt.subplots()
ax2.set(xlabel="Predicted Life Expectancy",
        ylabel="Residuals (Actual - Predicted Life Expectancy)")
ax2.scatter(x=y_hat, y=y-y_hat, color="blue", alpha=0.2);

In [ ]:
df.head()

In [ ]:
label_encoder = LabelEncoder()
status_labels = label_encoder.fit_transform(df['DistrictName'])
status_labels

In [ ]:
df = df.dropna().reset_index(drop = True)
df = df[df['SalePrice'] != 0.0]
df = df[df['SqFt'] != 0.0]

In [ ]:
df.corr()

In [ ]:
# # create a smaller df to save space and processing power
fsm_df = df[['PropertyClass', 'NbrThisType', 'SqFt', 'NbrBaths', 'SalePrice']].copy()
fsm_df.dropna(inplace=True)
fsm = ols(formula="SalePrice ~ PropertyClass + NbrThisType + SqFt + NbrBaths", data=fsm_df)
fsm_results = fsm.fit()
fsm_results.summary()

In [ ]:
df.head()

In [ ]:
df['DistrictName'].value_counts()

In [30]:
df_merged = consolidate_data(year=2019, create=True)
cols = list(df_merged.columns)
# cols = cols[2:4] + cols[6:7] + cols[10:11] + cols[27:29] + cols[35:36] + cols[43:44] + cols[48:50] 
cols = cols[2:4] +  cols[6:7] +  cols[10:11] + cols[27:29] + cols[35:36] + cols[43:44] + cols[48:50] 
df = df_merged[cols]
df.isna().sum()

df_merged.to_csv ('~\Downloads\test.csv', index = False, header=True)

Done eading Sales data.... (41818, 6)
Before EXTR_Parcel.csv:  (616089, 81)
After filtering KING county rows (103217, 27)
Filtering Residential and Condo data.... (98156, 27)
After reading EXTR_ResBldg.csv:  (517554, 30)
Done reading EXTR_LookUP.csv:  (1208, 3)
Merging....
After Merging files.csv:  (98156, 26)
Created merged file...s
Merging....Done


OSError: [Errno 22] Invalid argument: 'C:\\Users\\awyeh\\Downloads\test.csv'

In [31]:
df = df_merged.copy()
df = df.dropna().reset_index(drop = True)
df.drop(columns = ['DistrictName', 'PropType'], inplace = True)
df.head()

,Merged_Key,DocumentDate,SalePrice,PropertyType,PrincipalUse,PropertyClass,Area,SubArea,SqFtLot,WaterSystem,...,SqFtDeck,HeatSystem,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,YrRenovated,PcntComplete
0,098400000450,2019,409950,11,6,8,51.0,6.0,7875.0,2.0,...,140.0,5.0,3.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0
1,797320002320,2019,540000,3,6,8,23.0,4.0,8621.0,2.0,...,0.0,5.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
2,082607009096,2019,930000,11,6,8,70.0,3.0,212911.0,1.0,...,0.0,5.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
3,410200000075,2019,379950,11,6,8,40.0,9.0,14149.0,1.0,...,520.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,868229001120,2019,620000,14,6,8,95.0,10.0,4046.0,2.0,...,0.0,5.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0


In [25]:
df_merged['PropType'].value_counts()

R    5191
Name: PropType, dtype: int64

In [29]:
df_merged.corr()

,SalePrice,PropertyType,PrincipalUse,PropertyClass,Area,SubArea,SqFtLot,WaterSystem,SewerSystem,Access,...,SqFtDeck,HeatSystem,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,YrRenovated,PcntComplete
SalePrice,1.000000,-0.031509,0.039604,-0.136652,0.056015,0.003919,-0.058341,0.047904,0.007288,-0.004308,...,0.054871,0.022847,0.122679,0.089523,0.081278,0.151194,0.030813,0.047779,0.029698,0.038642
PropertyType,-0.031509,1.000000,0.104248,0.060668,0.029704,-0.026820,-0.055920,-0.117649,-0.142361,-0.069298,...,0.014483,0.021402,0.014308,0.015048,0.009584,0.018897,0.018267,-0.000424,0.000547,0.052638
PrincipalUse,0.039604,0.104248,1.000000,0.681831,0.008415,-0.019348,0.001858,-0.017730,0.004611,-0.037159,...,-0.008131,-0.007891,-0.002233,-0.023937,-0.014702,0.007912,-0.038322,-0.019841,-0.009418,0.014188
PropertyClass,-0.136652,0.060668,0.681831,1.000000,-0.045795,0.032028,-0.001385,0.248853,0.207039,0.202249,...,0.031153,0.004801,-0.017773,0.024158,0.017071,-0.031316,0.029470,0.045530,0.022473,-0.143556
Area,0.056015,0.029704,0.008415,-0.045795,1.000000,-0.004247,0.147667,-0.315931,-0.304120,-0.135903,...,0.125414,-0.120637,-0.205892,0.055916,-0.040797,0.041108,0.034746,0.031749,0.057901,-0.013718
SubArea,0.003919,-0.026820,-0.019348,0.032028,-0.004247,1.000000,-0.047573,0.130289,0.006633,0.014234,...,-0.028974,0.025243,0.040728,0.114400,-0.065637,0.096217,0.087910,-0.000604,-0.022291,-0.041509
SqFtLot,-0.058341,-0.055920,0.001858,-0.001385,0.147667,-0.047573,1.000000,-0.328105,-0.230581,0.021971,...,-0.073097,-0.471347,-0.416659,-0.106533,-0.096850,-0.253623,-0.011260,-0.066792,-0.021503,-0.006197
WaterSystem,0.047904,-0.117649,-0.017730,0.248853,-0.315931,0.130289,-0.328105,1.000000,0.402566,0.331619,...,-0.095249,0.256601,0.300427,0.087218,0.036973,0.166528,0.088928,0.062368,-0.064168,-0.034188
SewerSystem,0.007288,-0.142361,0.004611,0.207039,-0.304120,0.006633,-0.230581,0.402566,1.000000,0.256709,...,-0.207680,0.158960,0.232783,0.030736,-0.005673,0.145831,0.066861,-0.094516,-0.066141,-0.027218
Access,-0.004308,-0.069298,-0.037159,0.202249,-0.135903,0.014234,0.021971,0.331619,0.256709,1.000000,...,-0.146642,-0.074482,0.009325,-0.070434,-0.078953,-0.060975,0.020265,-0.028827,-0.046188,-0.018717


In [ ]:
# # create a smaller df to save space and processing power
fsm_df = df_merged[['SalePrice', 'SqFt1stFloor', 'SqFt2ndFloor', 'SqFtTotLiving', 'SqFtGarageAttached','SqFtGarageAttached','SqFtOpenPorch','SqFtEnclosedPorch','Bedrooms','BathHalfCount','BathFullCount']].copy()
fsm_df.dropna(inplace=True)
fsm = ols(formula="SalePrice ~ SqFt1stFloor + SqFt2ndFloor + SqFtTotLiving + SqFtGarageAttached + SqFtGarageAttached + SqFtOpenPorch + SqFtEnclosedPorch + Bedrooms + BathHalfCount + BathFullCount", data=fsm_df)
fsm_results = fsm.fit()
fsm_results.summary()